<a href="https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/2_zaklady-pythonu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Úvod a cíle kapitoly

V této kapitole se podíváme na získání dat z různých internetových platforem.  Webový prohlížeč je jen jeden ze způsobů, jak využívat internet. Mnohé aplikace, které máme v našich počítačích i chtrých telefonech, získávají z internetu data, aniž by webový prohlížeč jakkoli vstupoval do hry. Python a Jupyter notebooky mohou být využitý jako takováto aplikace. K mnohým datům, která jsou dostupná na internetu, se tak lze dostat přímo z našeho Python prostředí, aniž bychom museli data nejprve stahovat a poté do Pythonu načítat ve formě souborů.

Tato data se z webu získávají typicky pomocí protokolu HTTP (Hypertext Transfer Protocol): Aplikace vznese dotaz v podobě konkrétní URL adresy, na který webovy server podá odpověď ve formě dat. Tento proces se vlastně děje i na pozadí našeho prohlížeče: Přes zadanou URL adresu doazujeme určitý webový server, který našemu prohlížeči jako odpověď vrátí tzv. zdrojový kód stránky ve formátu HTML. Vyzkoušejme si to nyní na několika příkladech.


Jako první příklad nám mohou posloužit webové stránky hesel ze Slovníku českých filozofů dostupná z tohoto rozcestníku: https://filozofie.phil.muni.cz/vyzkum/publikace/scf/abecedni-seznam

In [9]:
# naimportujeme si několik knihoven
import requests # python knihovna pro vznášení HTTP dotazů
from bs4 import BeautifulSoup # python knihovna pro práci s daty ve formátu html či xml
import pandas as pd

In [24]:
# url adresa hesla věnovaného Jiřímu Fialovi:
url = "https://www.phil.muni.cz/fil/scf/komplet/fiala.html"

In [25]:
# na tuto webovou adresu nyní vzneseme dotaz pomocí knihovny requests
# odpověď si uložíme do proměnné `resp`:
resp = requests.get(url)

In [26]:
# odpověď si vypíšeme
resp

<Response [200]>

Pokud máme funkční připojení k internetu a a validní URL adresu, odpověď by měla být: "<Response [200]>".
Avšak v případě, že jsme například zvolili neexistující URL, odpověď bude jiná. Vyzkoušejme:

In [27]:
url = "https://www.phil.muni.cz/fil/scf/komplet/plato.html"
resp_test = requests.get(url)
resp_test

<Response [404]>

Vraťme se však zpět k naší odpovědi (objektu `resp`) z funkční URL adresy. Tento objekt totiž v sobě nese mnohem více, než jen informaci o tom, zda jsme obdrželi validní odpověď. Tato další data se skrývají zejména pod atributem `.text` (či `.content`)

In [41]:
url = "https://www.phil.muni.cz/fil/scf/komplet/fiala.html"
resp = requests.get(url)
resp.text

'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML//EN">\r\n<html>\r\n\r\n\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=windows-1250">\r\n<meta name="Script" content="tex2html">\r\n\r\n\r\n<title>Jiøí Fiala</title>\r\n</head>\r\n\r\n\r\n<body bgcolor="#FFFFFF">\r\n\r\n<p align="right"><a href="../../index.html" target="_top"><img src="../../gif/kat.gif" align="right" border="0" hspace="0" width="69" height="73"></a></p>\r\n\r\n<p><font size="4"><strong>Jiøí Fiala</strong></font></p>\r\n\r\n<p><p><font size="3"><strong>* 24. 2. 1939 Uherské Hradi\x9atì</strong><br>\r\n</font>\r\n<p><p><font size="3"><strong>\x86 22. 11. 2012 Brno</strong><br>\r\n</font>\r\n <p><p><img src="../foto/fiala.jpg" align="left" hspace="6" width="150" height="175"></p>  \r\nPo studiu matematiky na PøF MU v Brnì (1956\x9661; RNDr. 1967) pùsobil v l. 1961\x9690 jako programátor u Ès. státních drah. \r\nOd r. 1990 pracoval na katedøe matematické logiky a filozofie matematiky MFF UK; v r. 1990 zí

Co zde vidíme? Jedná se o zdrojový kód příslušné webové. Přesně z těchto dat náš prohlížeč vychází, když nám prezentuje vybranou webovou stránku. V tomto kroku je daný kód zapouzdřený jako proměnná typu string. Všimněme si, že zde máme problém se znakovou sadou: např. "pøekladù" namísto "překladů". Tento nedostatek však rychle napravíme a na text se podíváma ještě jednou: 


In [43]:
resp.encoding = 'windows-1250' # volba znakové sady
resp.text

'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML//EN">\r\n<html>\r\n\r\n\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=windows-1250">\r\n<meta name="Script" content="tex2html">\r\n\r\n\r\n<title>Jiří Fiala</title>\r\n</head>\r\n\r\n\r\n<body bgcolor="#FFFFFF">\r\n\r\n<p align="right"><a href="../../index.html" target="_top"><img src="../../gif/kat.gif" align="right" border="0" hspace="0" width="69" height="73"></a></p>\r\n\r\n<p><font size="4"><strong>Jiří Fiala</strong></font></p>\r\n\r\n<p><p><font size="3"><strong>* 24. 2. 1939 Uherské Hradiště</strong><br>\r\n</font>\r\n<p><p><font size="3"><strong>† 22. 11. 2012 Brno</strong><br>\r\n</font>\r\n <p><p><img src="../foto/fiala.jpg" align="left" hspace="6" width="150" height="175"></p>  \r\nPo studiu matematiky na PřF MU v Brně (1956–61; RNDr. 1967) působil v l. 1961–90 jako programátor u Čs. státních drah. \r\nOd r. 1990 pracoval na katedře matematické logiky a filozofie matematiky MFF UK; v r. 1990 získal hodnost

Nyní je již vše správně. Tato data v sobě však nesou více než prostý text - jsou ve formátu HTML, sestávající z mnoha tagů, které určují kde začíná a končí řádka, vyznačují další hypertextové odkazy apod. Nyní proto použijeme knihovny `BeautifulSoup`, abychom tento text interpretovali (parsovali) jako HTML kód, ve kterém se lze pohybovat jako v jakémsi stromu:

In [46]:
soup = BeautifulSoup(resp.text, "html.parser")

Objekt soup v sobě skrývá navigovatelnou stromovou strukturu celé html stránky. Můžeme zde nyní přistupovat k datům z jednotlivých tagů případně je i měnit. Názorné příklady jsou k dispozici v dokumentaci knihovny v sekci "Quick start". 

Například pod tagem `title` se skrývá název příslušné webové stránky:

In [48]:
print(soup.prettify())

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML//EN">
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="tex2html" name="Script"/>
  <title>
   Jiří Fiala
  </title>
 </head>
 <body bgcolor="#FFFFFF">
  <p align="right">
   <a href="../../index.html" target="_top">
    <img align="right" border="0" height="73" hspace="0" src="../../gif/kat.gif" width="69"/>
   </a>
  </p>
  <p>
   <font size="4">
    <strong>
     Jiří Fiala
    </strong>
   </font>
  </p>
  <p>
   <p>
    <font size="3">
     <strong>
      * 24. 2. 1939 Uherské Hradiště
     </strong>
     <br/>
    </font>
    <p>
     <p>
      <font size="3">
       <strong>
        † 22. 11. 2012 Brno
       </strong>
       <br/>
      </font>
      <p>
       <p>
        <img align="left" height="175" hspace="6" src="../foto/fiala.jpg" width="150"/>
       </p>
       Po studiu matematiky na PřF MU v Brně (1956–61; RNDr. 1967) působil v l. 1961–90 jako programátor u Čs. státních drah

In [81]:
soup.title

<title>Jiří Fiala</title>

Pokud nás zajímá pouze textový obsah daného tagu, příkaz ještě doplníme o atribut `string` nebo metodu `get_text()`

In [82]:
soup.title.string

'Jiří Fiala'

Po troše experimentování a prozkoumávání můžeme například zjistit, že datum narození a úmrtí se v dané struktuře nachází zde:

In [85]:
soup.body.find_all("p")[2].find_all("strong")[0].string

'* 24. 2. 1939 Uherské Hradiště'

In [86]:
soup.body.find_all("p")[2].find_all("strong")[1].string


'† 22. 11. 2012 Brno'

To jsou velice zajímavá a cenná metadata. Je stejná struktura zachována i na dalších stránkách ve slovníku? To nyní prověříme.

In [101]:
url = "https://www.phil.muni.cz/fil/scf/komplet/machjs.html"
resp = requests.get(url)
resp.encoding = 'windows-1250'
soup = BeautifulSoup(resp.text, "html.parser")
soup.title.string # vypíšeme název html

'Josef Macháček'

In [102]:
soup.body.find_all("p")[2].find_all("strong")[0].get_text() # pokusíme se nalézt datum a místo narození

'* 4. \r\n12. \r\n1917 Brno'

In [104]:
soup.body.find_all("p")[2].find_all("strong")[1].get_text() # pokusíme se nalézt datum a místo úmrtí

'† 12. \r\n7. \r\n1984 Brno'

Práce s mnoha webovými stránkami najednou

Webové stránky Slovníku českých filozofů mají abecední rozcestník: https://filozofie.phil.muni.cz/vyzkum/publikace/scf/abecedni-seznam. Při ohledání zdrojového kódu této stránky se ukazuje, že obsahuje URL adresy všech hesel ve slovníku. Srdce datového analytika zaplesá: To znamená, že můžeme jednu po druhé procházet všechny dané webové stránky a získat z nich data, která nás zajímají, například informace o datech a místech narození a úmrtí všech zahrnutých osobností. 

Nejprve budeme postupovat stejně jako výše: Pomocí `requests` získáme objekt `resp`, jehož datový obsah v podobě zdrojového kódu příslušné webové adresy naparsujeme pomocí `BeautifulSoup()` do navigovatelného stromu. 

In [96]:
url = "https://filozofie.phil.muni.cz/vyzkum/publikace/scf/abecedni-seznam"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")

Nyní pomocí cyklu FOR projedeme všechny tagy `a` (stanardní tag pro hypertextový odkaz). Pokud se v hodnotě atributu "href" tohoto tagu nachází URL adresa začínající stejně jako URL adresy jednotlivých hesel ve slovníků, extrahujeme tuto adresu a přidáme ji na seznam `filtered_hrefs`. (Ve zdrojovém kódu se může vyskytovat mnoho dalších URL adres, odkazujících např. kamsi na web Masarykovy univerzity, ty nás ale nyní nezajímají, proto tato podmínka.)

In [97]:
filtered_hrefs = []

for a_tag in soup.find_all('a'):
    href = a_tag.get('href')    # Extract 'href' attribute value.
    if href and href.startswith('http://www.phil.muni.cz/fil/scf/komplet/'):   # condition
        filtered_hrefs.append(href)

['http://www.phil.muni.cz/fil/scf/komplet/adam.html', 'http://www.phil.muni.cz/fil/scf/komplet/adamik.html', 'http://www.phil.muni.cz/fil/scf/komplet/albert.html', 'http://www.phil.muni.cz/fil/scf/komplet/albik.html', 'http://www.phil.muni.cz/fil/scf/komplet/alexej.html', 'http://www.phil.muni.cz/fil/scf/komplet/altr.html', 'http://www.phil.muni.cz/fil/scf/komplet/amerl.html', 'http://www.phil.muni.cz/fil/scf/komplet/armbrs.html', 'http://www.phil.muni.cz/fil/scf/komplet/arnold.html', 'http://www.phil.muni.cz/fil/scf/komplet/ariaga.html', 'http://www.phil.muni.cz/fil/scf/komplet/augolm.html', 'http://www.phil.muni.cz/fil/scf/komplet/babak.html', 'http://www.phil.muni.cz/fil/scf/komplet/bakos.html', 'http://www.phil.muni.cz/fil/scf/komplet/balabn.html', 'http://www.phil.muni.cz/fil/scf/komplet/balazo.html', 'http://www.phil.muni.cz/fil/scf/komplet/balbin.html', 'http://www.phil.muni.cz/fil/scf/komplet/balej.html', 'http://www.phil.muni.cz/fil/scf/komplet/banovs.html', 'http://www.phil.m

vypišme si prvních 10 URL adres:

In [98]:
filtered_hrefs[:10]

['http://www.phil.muni.cz/fil/scf/komplet/adam.html',
 'http://www.phil.muni.cz/fil/scf/komplet/adamik.html',
 'http://www.phil.muni.cz/fil/scf/komplet/albert.html',
 'http://www.phil.muni.cz/fil/scf/komplet/albik.html',
 'http://www.phil.muni.cz/fil/scf/komplet/alexej.html',
 'http://www.phil.muni.cz/fil/scf/komplet/altr.html',
 'http://www.phil.muni.cz/fil/scf/komplet/amerl.html',
 'http://www.phil.muni.cz/fil/scf/komplet/armbrs.html',
 'http://www.phil.muni.cz/fil/scf/komplet/arnold.html',
 'http://www.phil.muni.cz/fil/scf/komplet/ariaga.html']

Nyní vytvoříme cyklus FOR, v rámci kterého získáme zdrojový kód HTML stránky každé z těchto URL adres pomocí `requests`, zpracujeme jej pomocí `BeautifulSoup` a pokusíme se získat data se jmény jednotlivých filozofů a o místech a datech narození a úmrtí. 

In [117]:
failed = []
slovnik_data = []
for url in filtered_hrefs[:]:
    try:
        resp = requests.get(url)
        resp.encoding = 'windows-1250'
        soup = BeautifulSoup(resp.text, "html.parser")
        title = soup.title.string
        birth = soup.body.find_all("p")[2].find_all("strong")[0].string
        try:
            death = soup.body.find_all("p")[2].find_all("strong")[1].string
        except:
            death = None
        if "*" not in str(birth):
            birth = None
        if "†" not in str(death):
            death = None
        slovnik_data.append({"url" : url, "name" : title, "birth" : birth, "death" : death})
    except:
        failed.append(url)

Z dat, která jsme takto získali, vytvoříme velice snadno tabulku, respektivě objekt typu `pandas.DataFrame`:

In [118]:
slovnik_df = pd.DataFrame(slovnik_data) # vytvoříme dataframe
slovnik_df # dataframe si vypíšeme

,url,name,birth,death
0,http://www.phil.muni.cz/fil/scf/komplet/adam.html,Daniel Adam z Veleslavína,* 31. \r\n8. \r\n1546 Veleslavín u Prahy,† 18. \r\n10. \r\n1599 Praha
1,http://www.phil.muni.cz/fil/scf/komplet/adamik...,Richard Adamík,* 4. \r\n4. \r\n1867 Hranice na Moravě,† 15. \r\n8. \r\n1952
2,http://www.phil.muni.cz/fil/scf/komplet/albert...,František Albert,* 29. \r\n4. \r\n1856 Žamberk,† 22. \r\n7. \r\n1923 Potštejn
3,http://www.phil.muni.cz/fil/scf/komplet/albik....,Albík z Uničova,* asi 1358 Uničov,† 1427 Uhry
4,http://www.phil.muni.cz/fil/scf/komplet/alexej...,Nikolaj Alexejev,* 1. 5. 1879 Moskva,† 2. 3. 1964 Ženeva
...,...,...,...,...
867,http://www.phil.muni.cz/fil/scf/komplet/zumr.html,Josef Zumr,* 19. 3. 1928 Úmyslovice u Poděbrad,None
868,http://www.phil.muni.cz/fil/scf/komplet/zuska....,Vlastimil Zuska,* 5. \r\n5. \r\n1951 Praha,None
869,http://www.phil.muni.cz/fil/scf/komplet/zverin...,Josef Zvěřina,* 3. \r\n5. \r\n1913 Střítež u Třebíče,† 18. \r\n8. \r\n1990 Netunno (Itálie)
870,http://www.phil.muni.cz/fil/scf/komplet/zykmnd...,Václav Zykmund,* 18. \r\n6. \r\n1914 Praha,† 10. \r\n5. \r\n1984 Brno


Vidíme relativně pěkně naformátovanou tabulku se čtyřmi sloupci. Některé hodnoty jsou prázdné (`None`), například ve sloupci "death" v případě, že se jedná o žijícího autora. V jiných případech se zdá, že některá hesla jsou naformátovaná odlišně.

 Z praktických důvodů jsme se však omezili pouze na 10 prvních hesel. Dropnou úpravou (odstraněním "[:10"]) však lze skript snadno přenastavit tak, aby se aplikoval na všechny URL adresy na seznamu. Tomu se však nyní společně vyhneme, abychom příslušnou webovou doménu zbytečně nezavalovali tisíci HTTP dotazy. Na místo toho si tato kompletní data načteme z místa, kam jsem je dopředu uložil:

In [123]:
slovnik_df = pd.read_csv("../data/slovnik_df.csv")
slovnik_df

,url,name,birth,death
0,http://www.phil.muni.cz/fil/scf/komplet/adam.html,Daniel Adam z Veleslavína,* 31. \r\n8. \r\n1546 Veleslavín u Prahy,† 18. \r\n10. \r\n1599 Praha
1,http://www.phil.muni.cz/fil/scf/komplet/adamik...,Richard Adamík,* 4. \r\n4. \r\n1867 Hranice na Moravě,† 15. \r\n8. \r\n1952
2,http://www.phil.muni.cz/fil/scf/komplet/albert...,František Albert,* 29. \r\n4. \r\n1856 Žamberk,† 22. \r\n7. \r\n1923 Potštejn
3,http://www.phil.muni.cz/fil/scf/komplet/albik....,Albík z Uničova,* asi 1358 Uničov,† 1427 Uhry
4,http://www.phil.muni.cz/fil/scf/komplet/alexej...,Nikolaj Alexejev,* 1. 5. 1879 Moskva,† 2. 3. 1964 Ženeva
...,...,...,...,...
867,http://www.phil.muni.cz/fil/scf/komplet/zumr.html,Josef Zumr,* 19. 3. 1928 Úmyslovice u Poděbrad,NaN
868,http://www.phil.muni.cz/fil/scf/komplet/zuska....,Vlastimil Zuska,* 5. \r\n5. \r\n1951 Praha,NaN
869,http://www.phil.muni.cz/fil/scf/komplet/zverin...,Josef Zvěřina,* 3. \r\n5. \r\n1913 Střítež u Třebíče,† 18. \r\n8. \r\n1990 Netunno (Itálie)
870,http://www.phil.muni.cz/fil/scf/komplet/zykmnd...,Václav Zykmund,* 18. \r\n6. \r\n1914 Praha,† 10. \r\n5. \r\n1984 Brno


In [5]:
soup = BeautifulSoup(resp.text)
print(soup.prettify())

<!DOCTYPE html>
<!--[if lte IE 9]>    <html lang="cs" class="old-browser no-js no-touch"> <![endif]-->
<!--[if IE 9 ]>    <html lang="cs" class="ie9 no-js no-touch"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html class="no-js" lang="cs">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="Masarykova univerzita" name="author"/>
  <title>
   Abecední seznam hesel | Katedra filozofie
  </title>
  <meta content="@muni_cz" name="twitter:site"/>
  <meta content="summary" name="twitter:card"/>
  <meta content="Oficiální stránky Katedry filosofie Filosofické fakulty Masarykovy univerzity. Informace pro uchazeče o studium, pro studenty i pedagogy, učební pomůcky, fotografie ze života společenství." name="description">
   <meta content="katedra, fakulta, filosofie, filozofie, univerzita, Brno, kritické myšlení, argumentace, bytí, poznání, pravda," name="keywords">
    <meta content="Abecední seznam hese

### API přístup 
Systém dotazování v podobě URL adres a odpovídání na ně v podobě dat je někdy standardizovaný do podoby tzv API: Application Programming Interface, kdy vracená data již nemají podobu webových stránek, ale specificky strukturovaných dat, nad kterými vývojář zajásá. Tímto způsobem se na našich chytrých telefonech aktualizují data o počasí (viz např. dokumentaci k API od [OpenWeatherMa](https://openweathermap.org/api/one-call-3#current)).  Aby však nedošlo k přetížení či zneužití těchto služeb, většina API vyžaduje nějakou formu autorizace, nejčastěji ve formě jakéhosi automaticky generovaného klíče či tokenu pro registrované uživatele.

Tato technologie je využívána i pro potřeby zpřístupňování dat z oblasti péče o kulturní dědictví. V této sekci si ukážeme několik příkladů těchto webových služeb.

Projekt Epigraphic Database Heidelberg hostí digitalizovanou kolekci více než 80,000 převážně latinských nápisů z antického Říma. Tyto nápisy si veřejnost může prohlížet a prohledávat za využití webových stránek [zde](https://edh.ub.uni-heidelberg.de/inschrift/suche). Kromě toho má však badatel ještě jinou možnost, jak se dostat k příslušným datům, a totiž právě za využití speciálně vyvinutého API, které je zdokumentováno [zde](https://edh.ub.uni-heidelberg.de/data/api).

V této dokumentaci se podrobně dočteme, jaké parametry můžeme v našem dotazu (query) použít a jak. 

Základní URL adresa je `https://edh.ub.uni-heidelberg.de/data/api/inschrift/suche?`, za níž připojujeme parametry pro určení námi hledané skupiny nápisů.

Chceme-li např. získat data o všech nápisech z Římské provincie *Germania superior*, nastavíme parametr "provinz" na hodnotu "ges". Výsledná URL adresa pak vypadá takto:

```
https://edh.ub.uni-heidelberg.de/data/api/inschrift/suche?provinz=ges
```